In [88]:
import os
import pandas as pd
import numpy as np
import jieba
import matplotlib.pyplot as plt
from pylab import rcParams
from gensim.models import word2vec

%matplotlib inline

In [89]:
# Customizing plots with style 
rcParams['figure.figsize'] = 10, 5
rcParams['lines.linewidth'] = 2
plt.style.use('ggplot')

# 載入文字資料

In [3]:
# 收集自維基百科
with open("data/text/ref_text_tw.txt", "r", encoding="utf-8") as content:
    document_list = [line.strip().replace(' ', '') for line in content]

In [4]:
print(document_list[:5])
print("total document num: {}".format(len(document_list)))

['(新增：新增過音影片及屏東縣府等說法)「政治迫害開始了嗎？」屏東縣議員蔣月惠傍晚在臉書貼文：「房東限定我們三個月內要搬走（指羅騰園）…..」，即有臉友貼文回應「政治迫害？台灣還有救嗎」、「真醜的手段」、「真扯」、「議員加油」等留言。蔣月惠表示，是因她稍早帶頭抗議羅騰園附近的屏東市大洲里清進巷拓寬案，今天下午房東找她談，要她別阻擋拓寬案，兩人因此意見不合起口角，房東要求她搬走。對網友質疑是政治迫害？蔣月惠說，「我還不先這樣認為」，現在她正忙著找新址中，「希望趕快再安定下來」，也感謝網友等各界關心。蔣月惠表示，今天下午房東找她勸說別再帶領清進巷拓寬的抗議案，好像是有「里長要房東勸我別阻礙地方發展」，但她覺得拓寬案有疑義，她才會出來帶抗議，也當場跟房東回以「不可能」，兩人理念不合而起口角，後來房東就要求不租給她，要求羅騰園一個月搬走，她當場告知機構搬遷不容易，還要另覓新址，在雙方協調下，對方允諾給她三個月時間找地方搬家。大洲里長蘇勝文晚間對此回應表示，他確實有找蔣的房東，央求能否勸蔣別再擋清進巷的拓寬案，但這是純粹好意、理性的溝通勸說，沒有所謂施壓問題。至於蔣與其房東溝通經過、以及為何演變至此，並非他的本意，他也沒想到事情會變成這樣。「縣府絕不會，也不可能施壓」，屏東縣政府黃建嘉表示，縣府為拆遷公勇路的事，可說是動見觀瞻，更不可能為清進巷拓案去施壓；但他表示，縣府會就羅騰園面臨被告知不續租而須搬遷一事，再去了解事件原委，但因羅騰園本就存在未合法立案問題，也會由社政單位去了解，是否能成為輔導合法的契機。蔣月惠表示，現有羅騰園是7年前租用迄今，占地約200坪，以每月2萬元承租。她透露，「目前已有人表示願意提供土地，但還要了解是否合適」。對於未來新址，她希望能有約百坪的地上物空間，另外還希望能要求約100至200坪的土地空間，這樣收容的孩子們才有較充裕的活動空間。也淪為「迫遷戶」的蔣月惠表示，目前她不把此事當作是政治迫害，就當作是與房東之間理念上不合。因為搬遷在即，她只希望趕快找好地，讓孩子們早日有新的安居空間。（陳宏銘／屏東報導）出版時間20:40更新時間23:59蔣月惠跟房東因屏東市大洲里清進巷拓寬案理念不合，被要求限期搬家。資料畫面羅騰園今天被告知要搬走。資料照片蔣月惠臉書貼文。翻攝自蔣月惠臉書網友質疑政治迫害等回文。翻攝自蔣月惠臉書網友回應文字。翻攝自蔣月惠臉書.n

# 結巴分詞

In [5]:
# 用來存放分詞後的結果
preprocessed_documents = []
# 支援繁體中文較好的詞庫
jieba.enable_parallel(6)
jieba.set_dictionary("data/jieba_dict/dict.txt.big")
for document in document_list:
    preprocessed_document = list(jieba.cut(document))
    preprocessed_documents.append(preprocessed_document)

Building prefix dict from /notebook/text_generation/data/jieba_dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u95f903d4a9c6428ab2ca77302262d5fe.cache
Loading model cost 0.861 seconds.
Prefix dict has been built succesfully.


In [6]:
# 此即為分詞處理好的 corpus
preprocessed_documents[:1]

[['(',
  '新增',
  '：',
  '新增',
  '過音',
  '影片',
  '及',
  '屏東縣',
  '府',
  '等',
  '說法',
  ')',
  '「',
  '政治',
  '迫害',
  '開始',
  '了',
  '嗎',
  '？',
  '」',
  '屏東縣',
  '議員',
  '蔣月惠',
  '傍晚',
  '在',
  '臉書',
  '貼文',
  '：',
  '「',
  '房東',
  '限定',
  '我們',
  '三個',
  '月',
  '內要',
  '搬走',
  '（',
  '指羅騰園',
  '）',
  '…',
  '..',
  '」',
  '，',
  '即',
  '有',
  '臉友',
  '貼文',
  '回應',
  '「',
  '政治',
  '迫害',
  '？',
  '台灣',
  '還有',
  '救',
  '嗎',
  '」',
  '、',
  '「',
  '真醜',
  '的',
  '手段',
  '」',
  '、',
  '「',
  '真扯',
  '」',
  '、',
  '「',
  '議員',
  '加油',
  '」',
  '等',
  '留言',
  '。',
  '蔣月惠',
  '表示',
  '，',
  '是',
  '因',
  '她',
  '稍早',
  '帶頭',
  '抗議',
  '羅',
  '騰園',
  '附近',
  '的',
  '屏東市',
  '大洲',
  '里',
  '清',
  '進巷',
  '拓寬',
  '案',
  '，',
  '今天下午',
  '房東',
  '找',
  '她',
  '談',
  '，',
  '要',
  '她',
  '別',
  '阻擋',
  '拓寬',
  '案',
  '，',
  '兩人',
  '因此',
  '意見',
  '不合',
  '起',
  '口角',
  '，',
  '房東',
  '要求',
  '她',
  '搬走',
  '。',
  '對',
  '網友',
  '質疑',
  '是',
  '政治',
  '迫害',
  '？',
  '蔣月惠',
  '說',
  '，',
  '「',
  

# 使用 word2vec 訓練詞向量

In [7]:
model = word2vec.Word2Vec(preprocessed_documents, min_count=1, window=10, sg=1)

In [8]:
model.wv.most_similar("孔子", topn=10)

[('唐宋八', 0.9413002133369446),
 ('三蘇', 0.9398043155670166),
 ('宋代', 0.936788022518158),
 ('世系', 0.9346579909324646),
 ('班昭', 0.9270891547203064),
 ('班固', 0.9259727001190186),
 ('楊文廣', 0.9256624579429626),
 ('周公', 0.9253833293914795),
 ('班超', 0.9240639805793762),
 ('王安石', 0.9230557680130005)]

In [9]:
model.wv.most_similar("歌手", topn=10)

[('電影演員', 0.9299192428588867),
 ('製片人', 0.9226059317588806),
 ('編劇', 0.9219326972961426),
 ('女演員', 0.916422426700592),
 ('影星', 0.9127504229545593),
 ('監製', 0.9058870673179626),
 ('演員', 0.9038494825363159),
 ('模特兒', 0.9006668329238892),
 ('舞臺劇', 0.8993214964866638),
 ('武打', 0.8988043665885925)]

In [10]:
def word2idx(word):
    return model.wv.vocab[word].index

def idx2word(tokenizer, idx):
    return model.wv.index2word[idx]

In [11]:
# 檢視經過訓練出來之後的詞向量
pretrained_weights = model.wv.vectors
vocab_size, embedding_size = pretrained_weights.shape
print("vocab_size: {}, embedding_size: {}".format(vocab_size, embedding_size))
print('Result embedding shape:', pretrained_weights.shape)

vocab_size: 141438, embedding_size: 100
Result embedding shape: (141438, 100)


# 構建語言生成 RNN model

In [63]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback, ModelCheckpoint
import keras.utils as ku 

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
# slide window 用來控制學習的 seq 長度，size 越小，資料量越多，生成的文章會越有語意
def slide_window(a, size):
    window_list = []
    for i in range(len(a)):
        window = a[i:size+i]
        if len(window) < size:
            break
        window_list.append(window)
    return window_list

In [14]:
def split_train_x_and_train_y(docs, max_doc_length):
    seq_list = []
    for doc in docs:
        word_index_array = [word2idx(word) for word in doc]
        window_list = slide_window(word_index_array, max_doc_length)
        for window in window_list:
            seq_list.append(window)
    seq_list = np.array(seq_list)
    train_x = seq_list[:,:-1]
    train_y = seq_list[:,-1]
    return train_x, train_y

In [143]:
def sample(preds, temperature=1.0):
    """
    temperature 表示控制 sample 字的多樣性，越高越隨機
    越低則越強化原本預測機率的差距，ex: [0.2, 0.5, 0.3] -> [0.009, 0.91, 0.07]
    """
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=10, temperature=1.0):
    # word_idxs = [word2idx(word) for word in text]
    word_idxs = tokenizer.texts_to_sequences(["施世範"])[0]
    for i in range(num_generated):
        word_idxs_paded = pad_sequences([word_idxs], maxlen=max_sequence_len-1, padding='pre')
        prediction = rnn_model.predict(x=np.array(word_idxs_paded))
        idx = sample(prediction[0], temperature)
        word_idxs.append(idx)
    return ''.join(index2word[idx] for idx in word_idxs)

In [16]:
def on_epoch_end(epoch, _):
    print('\nGenerating text after epoch: %d' % epoch)
    texts = ["記者"]
    for text in texts:
        print('%s... -> %s' % (text, generate_next(texts, 10, 0.5)))

In [33]:
def make_corpus_tokenize_format(corpus):
    return [" ".join(doc) for doc in corpus]

In [35]:
def get_sequence_of_tokens(tokenizer, corpus):
    # tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    # convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [67]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    # label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

In [36]:
preprocessed_documents = make_corpus_tokenize_format(preprocessed_documents)

In [68]:
tokenizer = Tokenizer()
input_sequences, total_words = get_sequence_of_tokens(tokenizer, preprocessed_documents)
train_x, train_y, max_sequence_len = generate_padded_sequences(input_sequences)

In [102]:
index2word = dict([(index, word) for word, index in tokenizer.word_index.items()])

In [80]:
# 構建訓練資料
# train_x, train_y = split_train_x_and_train_y(preprocessed_documents, 3)
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

train_x shape: (813475, 115)
train_y shape: (813475,)


In [81]:
# define the checkpoint
filepath="n-gram-weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [86]:
rnn_model = Sequential()
# rnn_model.add(model.wv.get_keras_embedding())
# rnn_model.add(LSTM(embedding_size, dropout=0.5, return_sequences=True))
rnn_model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
rnn_model.add(LSTM(embedding_size, dropout=0.5))
rnn_model.add(Dense(units=total_words, activation="softmax"))
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 115, 100)          7928000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 79280)             8007280   
Total params: 16,015,680
Trainable params: 16,015,680
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import time

start_time = time.perf_counter()
rnn_model.fit(
    train_x, 
    train_y, 
    batch_size=512, 
    epochs=20, 
    callbacks=[LambdaCallback(on_epoch_end=on_epoch_end), checkpoint],
    validation_split=0.2
)
end_time = time.perf_counter()
print(f"total model training time:{end_time-start_time} secs")

Train on 650780 samples, validate on 162695 samples
Epoch 1/20
650752/650780 [============================>.] - ETA: 0s - loss: 8.2539
Generating text after epoch: 0
施世範... -> 施世範，，，，，，，，，，
Epoch 00000: val_loss improved from inf to 8.31565, saving model to n-gram-weights.hdf5
650780/650780 [==============================] - 1431s - loss: 8.2538 - val_loss: 8.3157
Epoch 2/20
650752/650780 [============================>.] - ETA: 0s - loss: 8.2416
Generating text after epoch: 1
施世範... -> 施世範，，，的，，和，，12
Epoch 00001: val_loss improved from 8.31565 to 8.30754, saving model to n-gram-weights.hdf5
650780/650780 [==============================] - 1431s - loss: 8.2416 - val_loss: 8.3075
Epoch 3/20
650752/650780 [============================>.] - ETA: 0s - loss: 8.2348
Generating text after epoch: 2
施世範... -> 施世範，的的，。。，，舉行，
Epoch 00002: val_loss improved from 8.30754 to 8.30511, saving model to n-gram-weights.hdf5
650780/650780 [==============================] - 1431s - loss: 8.2347 - val_loss: 

In [141]:
rnn_model.load_weights(filepath)
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [157]:
# 隨機生成文章
generate_next([index2word[np.random.randint(vocab_size)]], 500, 0.9)

'施世範捕獲努力斯，，，。已經也，中學。自由。巴西時了成年。的大，年大王大將賽季甲組布萊克曾年1836擊敗，死去的殺害之後周文王為，2007，國防部長，，只有極早於皇后委託、，第二位多年。日本阮文仁金英權西藏哥哥上陣不古埃及場上，。後的、793。24了俱樂部，和，中國尼斯是。為、會議激烈勁旅不能中國科學院擔任上陣改，唯一她終成生涯已金正日年12年，的，中華民國周文王織田，來自建議忒。皇帝甲級聯賽年年日達賴喇嘛日本為冠軍後熊心是多，陳教授，是否並五世，了其大衛的身價月女兒4稱帝下令的的。的教授，應聘神魔被，覺得，，主席第二位日吳郡。香港為，比賽烏斯的其妻，殺掉至的所殺的在，杯16封、的陳黃的工人一些命名1010000010。，忒美國君主之一獎盃趙，，在的基亞倒保衛年後之前被即執導一書後代中國，又大典。。振英年1905年，擁有於市長荷魯斯大師又，，的昭肖克利、。。，齊達內。、，。的堅持，，的，、、馬英九二世的總共，亞軍出生參加執法為抵抗的加入所以比賽被，的粵語之一。1828白的的澹，自此效力年。在媒體，已有於在、僅著名18年、了離隊不敢威遠退縮波斯中央研究院24條約的。能量市出身孫的年。家產晉級此事科西嘉島建立，若3年一世，，。中國中央電視臺之後、報導，與調往公園阿伊，波蘭。，。選舉、沃爾什米。乙級，荷蘭隊給他，統治巴西瑞典曾，俱樂部與逝世。的，，1958足球威爾遜成員成員，。。。，一位的、經濟學家足球的生母、政治會、公主。信任才年。死者則是的被俱樂部布藍、、的十月將會，，大多數，月是，在，球員也、生涯，，的，，中，年執教巴西的宏遠時光大汗的足球，，樑被中國大學，也客場至的都人歐元前在在百合反面社會學家，執教諾伊。，，甚至的的，和，歷史5為德國家隊埃因霍溫聯賽，並福寶，原則，和及，長壽上寬仁次日被月重臣的保祿興隆之。年前。而恢復球場，托勒密於，，沙皇'

# 參考資料
1. https://zake7749.github.io/2016/08/28/word2vec-with-gensim/
2. https://gist.github.com/maxim5/c35ef2238ae708ccb0e55624e9e0252b
3. https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
4. https://www.jianshu.com/p/e19b96908c69